In [66]:
import requests
import json
import pandas as pd

In [67]:
# get game id for majora's mask

games = requests.get('https://www.speedrun.com/api/v1/games?abbreviation=mm').text
game_dict = json.loads(games)['data'][0]

game_id = game_dict['id']
game_dict['names']

{'international': "The Legend of Zelda: Majora's Mask",
 'japanese': 'ゼルダの伝説 ムジュラの仮面',
 'twitch': "The Legend of Zelda: Majora's Mask"}

In [68]:
# get all runs done for mm (max 200 at a time)

runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id).text
runs_dict1 = json.loads(runs)
runs_list = json.loads(runs)['data']
runs_dict1['pagination']['size'] # max elements



200

In [69]:
# next course of action:
# loop through each run (dict)
# get the run's player id and primary_t (time) add add to dictionary 'id':'time'

# how to get player id of run at list index 0
runs_list[0]['players'][0]['id']

# how to get time of run 0
runs_list[0]['times']['primary_t']

len(runs_list)

200

In [70]:
# create a function which loops through a page (200 runs) then adds those runs
# to a larger list and continues until all runs are in the massive list

def make_list():
  full_list = []
  offset = 0
  runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id).text
  dict = json.loads(runs)

  while dict['pagination']['size'] == 200:
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id + '&offset=' + str(offset)).text
    dict = json.loads(runs)
    full_list.extend(dict['data'])
    offset += 200

  return full_list

In [71]:
# create a method that converts an id to a username

def id_to_username(id):
  user_info = json.loads(requests.get('https://www.speedrun.com/api/v1/users/' + id).text)
  if user_info.get('data'): # not sure why data key errors are happening here
    username = user_info['data']['names']['international']
  else:
    return 'fail'

  return username

In [72]:
# create a function which loops through one page/request(to start) and adds the
# time of all runs to the user's id in dictionary
# note this method takes into account players who don't have src accounts and
# thus do not have an id, however they are not accounted for in the final return

ids_test = set()

# takes
def add_time():
  runs = make_list()
  player_times = {}
  id_names = {}
  final = {}
  ids = set()

  for i in range(len(runs)):
    if runs[i]['players'][0].get('id'):
      player = runs[i]['players'][0]['id']
      #player = id_to_username(player)
      ids.add(player)
    else:
      player = runs[i]['players'][0]['name']


    time = runs[i]['times']['primary_t']

    if player in player_times:
      player_times[player] += time
    else:
      player_times[player] = time

  # create a mapping of ids to usernames
  for id in ids:
    id_names[id] = id_to_username(id)

  for id in id_names.keys() & player_times.keys():
    final[id_names[id]] = player_times[id]

  return final




In [73]:
final_data = add_time()
#list(map(id_to_username, ids_test))
#ids_test

In [74]:
sorted = pd.Series(final_data).sort_values(ascending=False)
sorted

TrevPerson         1229098.00
fullgrowngaming     976611.00
dopeZera            700351.00
popesquidward       553996.81
EnNopp112           551924.47
                      ...    
kemXtrm               1780.00
BetoKos               1595.00
Marthur                759.00
Perzyk                 133.00
agdf_ydy               102.00
Length: 390, dtype: float64

In [75]:

def convert2(seconds):
  m, s = divmod(seconds, 60)
  h, m = divmod(m, 60)

  return str(int(h)) + ':' + str(int(m)) + ':' + str(int(s))

In [76]:
final = sorted.apply(convert2)

final.head(20)

TrevPerson         341:24:58
fullgrowngaming    271:16:51
dopeZera           194:32:31
popesquidward      153:53:16
EnNopp112          153:18:44
MajinPhil          135:17:32
NicoSpeedsub        133:1:15
clairelynnd        118:39:26
chopperman9        113:21:39
jess57329          102:33:42
HyrulianKnight      92:29:56
FFerraz1            86:18:48
AntiLink            83:11:34
bewildebeest        82:14:42
TrulyNaN             82:6:42
Draconif             75:2:47
Zunar               69:21:26
Butterlord           69:11:8
VPP                 67:49:40
SampleName_         65:35:16
dtype: object

In [82]:
test = json.loads(requests.get('https://www.speedrun.com/api/v1/users/1xy90mjr').text)
final['Your_Bro_Bri']


'14:56:42'

In [ ]:
runs

'{"data":[{"id":"8y89d7nm","weblink":"https://www.speedrun.com/oot/run/8y89d7nm","game":"j1l9qz1g","level":null,"category":"q255jw2o","videos":{"links":[{"uri":"http://www.twitch.tv/dennisbalow/c/5558982"}]},"comment":"Yeah subbed the 5. 5th try Dample, god like Fishing. Good movement overall, really low mistakes. I am super satisfied. :)","status":{"status":"verified","examiner":"e8e5v680","verify-date":null},"players":[{"rel":"user","id":"18vyq58l","uri":"https://www.speedrun.com/api/v1/users/18vyq58l"}],"date":"2014-11-24","submitted":"2015-02-17T03:52:08Z","times":{"primary":"PT4H57M15S","primary_t":17835,"realtime":"PT4H57M15S","realtime_t":17835,"realtime_noloads":null,"realtime_noloads_t":0,"ingame":null,"ingame_t":0},"system":{"platform":"w89rwelk","emulated":false,"region":"o316x197"},"splits":null,"values":{"jlzw5w7l":"0133d8y1"},"links":[{"rel":"self","uri":"https://www.speedrun.com/api/v1/runs/8y89d7nm"},{"rel":"game","uri":"https://www.speedrun.com/api/v1/games/j1l9qz1g"},